In [2]:
using DataFrames
using Statistics
using Dates
using XLSX

In [3]:
# λ = 0.70931715
λ = 1.5405929
ΔtthD = 0.005
px = 0.1353
standards = Dict(
    "Si_small" => 5.431042,
    "Ge" => 5.65789,
    "LaB6" => 4.15692,
    "BGO_40" => 10.5182,
    "BGO_O4B" => 10.148,
    "Y2O3_O4B" => 10.6046,
    "YEu_AM-17-22" => 10.6174,
)

Dict{String, Float64} with 7 entries:
  "LaB6"         => 4.15692
  "BGO_40"       => 10.5182
  "Ge"           => 5.65789
  "BGO_O4B"      => 10.148
  "YEu_AM-17-22" => 10.6174
  "Si_small"     => 5.43104
  "Y2O3_O4B"     => 10.6046

In [4]:
function bragg_tth(h, k, l)
    global λ, a0
    2asind(λ * hypot(h, k, l) / 2a0)
end

bragg_tth (generic function with 1 method)

In [5]:
function bond_calc(h, k, l, tth0, D, tthDp, tthDn, xp, xn)
    global λ , ΔtthD, px, a0
    γ = rad2deg(px / D) * cosd((tthDp + tthDn)/2)
    tth = abs(rem((tthDp - tthDn) / 2, 360, RoundNearest)) - γ * (xp - xn) / 2
    a = hypot(h, k, l) * λ / 2sind(tth / 2)
    tth, tth - tth0, a, a - a0
end

bond_calc (generic function with 1 method)

In [6]:
function hklpos(h, k, l)
    h != 0 ? h > 0 :
    k != 0 ? k > 0 :
    l > 0
end

hklpos (generic function with 1 method)

In [7]:
name = "YEu_AM-17-22"
a0 = standards[name]
data = DataFrame(XLSX.readtable("$(name).xlsx", name))
select!(data, :orient, :frame, :created, :distance, :tthD, :omega, :phi, :h, :k, :l, :x1)
transform!(data, [:h, :k, :l] => ByRow(bragg_tth) => :tth0)

Row,orient,frame,created,distance,tthD,omega,phi,h,k,l,x1,tth0
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64
1,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_01_0001.sfrm,2025-06-15T10:33:26,96.3,-80.0,-94.0,-124.6,-6,8,-4,681.643,102.776
2,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_02_0001.sfrm,2025-06-15T10:44:06,96.3,-80.0,86.0,-124.6,6,-8,4,681.411,102.776
3,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_03_0001.sfrm,2025-06-15T10:56:59,96.3,-79.9,86.0,-124.6,6,-8,4,682.828,102.776
4,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_01_0001.sfrm,2025-06-15T11:13:57,96.3,126.2,-61.0,-123.7,4,-6,-8,689.559,102.776
5,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_02_0001.sfrm,2025-06-15T11:24:36,96.3,126.2,119.0,-123.7,-4,6,8,689.794,102.776
6,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_03_0001.sfrm,2025-06-15T11:34:47,96.3,126.1,119.0,-123.7,-4,6,8,688.368,102.776


In [8]:
bond = innerjoin(data[data.tthD .> 0, :], data[data.tthD .< 0, :],
    on = [:distance, :tth0, :orient],
    renamecols = "_pos" => "_neg",
)
subset!(bond, [:created_pos, :created_neg] => ByRow((c1, c2) -> abs(c1 - c2) < Day(1)))
subset!(bond, [:phi_pos, :phi_neg] => ByRow((p1, p2) -> abs(rem(p1 - p2, 360, RoundNearest)) < 15))
subset!(bond, [:x1_pos, :x1_neg] => ByRow((x1, x2) -> abs(x1 - x2) < 10))
transform!(bond, [:h_pos, :k_pos, :l_pos, :tth0, :distance, :tthD_pos, :tthD_neg, :x1_pos, :x1_neg] => ByRow(bond_calc) => [:tth, :Δtth, :a, :Δa])

Row,orient,frame_pos,created_pos,distance,tthD_pos,omega_pos,phi_pos,h_pos,k_pos,l_pos,x1_pos,tth0,frame_neg,created_neg,tthD_neg,omega_neg,phi_neg,h_neg,k_neg,l_neg,x1_neg,tth,Δtth,a,Δa
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Float64,Float64,Float64
1,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_01_0001.sfrm,2025-06-15T11:13:57,96.3,126.2,-61.0,-123.7,4,-6,-8,689.559,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_01_0001.sfrm,2025-06-15T10:33:26,-80.0,-94.0,-124.6,-6,8,-4,681.643,102.807,0.030606,10.6151,-0.00226387
2,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_02_0001.sfrm,2025-06-15T11:24:36,96.3,126.2,119.0,-123.7,-4,6,8,689.794,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_01_0001.sfrm,2025-06-15T10:33:26,-80.0,-94.0,-124.6,-6,8,-4,681.643,102.798,0.0219149,10.6158,-0.00162118
3,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_03_0001.sfrm,2025-06-15T11:34:47,96.3,126.1,119.0,-123.7,-4,6,8,688.368,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_01_0001.sfrm,2025-06-15T10:33:26,-80.0,-94.0,-124.6,-6,8,-4,681.643,102.801,0.0246152,10.6156,-0.00182087
4,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_01_0001.sfrm,2025-06-15T11:13:57,96.3,126.2,-61.0,-123.7,4,-6,-8,689.559,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_02_0001.sfrm,2025-06-15T10:44:06,-80.0,86.0,-124.6,6,-8,4,681.411,102.798,0.0220025,10.6158,-0.00162766
5,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_02_0001.sfrm,2025-06-15T11:24:36,96.3,126.2,119.0,-123.7,-4,6,8,689.794,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_02_0001.sfrm,2025-06-15T10:44:06,-80.0,86.0,-124.6,6,-8,4,681.411,102.79,0.0133113,10.6164,-0.000984825
6,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_03_0001.sfrm,2025-06-15T11:34:47,96.3,126.1,119.0,-123.7,-4,6,8,688.368,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_02_0001.sfrm,2025-06-15T10:44:06,-80.0,86.0,-124.6,6,-8,4,681.411,102.792,0.0160084,10.6162,-0.00118433
7,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_01_0001.sfrm,2025-06-15T11:13:57,96.3,126.2,-61.0,-123.7,4,-6,-8,689.559,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_03_0001.sfrm,2025-06-15T10:56:59,-79.9,86.0,-124.6,6,-8,4,682.828,102.801,0.0245577,10.6156,-0.00181662
8,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_02_0001.sfrm,2025-06-15T11:24:36,96.3,126.2,119.0,-123.7,-4,6,8,689.794,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_03_0001.sfrm,2025-06-15T10:56:59,-79.9,86.0,-124.6,6,-8,4,682.828,102.792,0.0158697,10.6162,-0.00117407
9,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_03_0001.sfrm,2025-06-15T11:34:47,96.3,126.1,119.0,-123.7,-4,6,8,688.368,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_03_0001.sfrm,2025-06-15T10:56:59,-79.9,86.0,-124.6,6,-8,4,682.828,102.795,0.0185665,10.616,-0.00137353


In [9]:
frid = innerjoin(subset(data, [:h, :k, :l] => ByRow(hklpos)), transform(data, [:h, :k, :l] .=> -, renamecols = false),
    on = [:tthD, :distance, :phi, :h, :k, :l, :tth0, :orient],
    renamecols = "" => "_frid"
)
subset!(frid, [:created, :created_frid] => ByRow((c1, c2) -> abs(c1 - c2) < Day(1)))
transform!(frid, [:x1, :x1_frid] => ByRow((x, xf) -> (x + xf) / 2) => :x1_mean)

Row,orient,frame,created,distance,tthD,omega,phi,h,k,l,x1,tth0,frame_frid,created_frid,omega_frid,x1_frid,x1_mean
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Float64
1,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_02_0001.sfrm,2025-06-15T10:44:06,96.3,-80.0,86.0,-124.6,6,-8,4,681.411,102.776,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_01_0001.sfrm,2025-06-15T10:33:26,-94.0,681.643,681.527
2,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_01_0001.sfrm,2025-06-15T11:13:57,96.3,126.2,-61.0,-123.7,4,-6,-8,689.559,102.776,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_02_0001.sfrm,2025-06-15T11:24:36,119.0,689.794,689.676


In [10]:
eccentr = innerjoin(frid[frid.tthD .> 0, :], frid[frid.tthD .< 0, :],
    on = [:distance, :tth0, :orient],
    renamecols = "_pos" => "_neg"
)
subset!(eccentr, [:created_pos, :created_neg] => ByRow((c1, c2) -> abs(c1 - c2) < Day(1)))
subset!(eccentr, [:phi_pos, :phi_neg] => ByRow((p1, p2) -> abs(rem(p1 - p2, 360, RoundNearest)) < 15))
subset!(eccentr, [:x1_mean_pos, :x1_mean_neg] => ByRow((x1, x2) -> abs(x1 - x2) < 10))
transform!(eccentr, [:h_pos, :k_pos, :l_pos, :tth0, :distance, :tthD_pos, :tthD_neg, :x1_mean_pos, :x1_mean_neg] => ByRow(bond_calc) => [:tth, :Δtth, :a, :Δa])

Row,orient,frame_pos,created_pos,distance,tthD_pos,omega_pos,phi_pos,h_pos,k_pos,l_pos,x1_pos,tth0,frame_frid_pos,created_frid_pos,omega_frid_pos,x1_frid_pos,x1_mean_pos,frame_neg,created_neg,tthD_neg,omega_neg,phi_neg,h_neg,k_neg,l_neg,x1_neg,frame_frid_neg,created_frid_neg,omega_frid_neg,x1_frid_neg,x1_mean_neg,tth,Δtth,a,Δa
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Float64,Float64,Float64,Float64
1,20250615_YEu_AM-17-22.p4p,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_01_0001.sfrm,2025-06-15T11:13:57,96.3,126.2,-61.0,-123.7,4,-6,-8,689.559,102.776,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_02_0001.sfrm,2025-06-15T11:24:36,119.0,689.794,689.676,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_02_0001.sfrm,2025-06-15T10:44:06,-80.0,86.0,-124.6,6,-8,4,681.411,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_01_0001.sfrm,2025-06-15T10:33:26,-94.0,681.643,681.527,102.798,0.0219587,10.6158,-0.00162442


In [11]:
XLSX.openxlsx("$(name).xlsx", mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, "$(name)")
    XLSX.writetable!(xf[1], data)
    sheet = XLSX.addsheet!(xf, "bond")
    XLSX.writetable!(sheet, bond)
    sheet = XLSX.addsheet!(xf, "frid")
    XLSX.writetable!(sheet, frid)
    sheet = XLSX.addsheet!(xf, "eccentr")
    XLSX.writetable!(sheet, eccentr)
end

In [85]:
std(eccentr.a) / mean(eccentr.a)

2.7049266336164482e-6

In [5]:
rad2deg(0.1353 / 96.5) / cosd((80 - 114.8)/2)

0.08418512260195862

In [26]:
assym_calc(8, 5, 3, 96.5, 114.8, 80, 603.76, 605.67)

rad2deg(px / D) / cosd((tthDp - tthDn) / 2) = 0.08418512260195862


(97.48039679208487, 0.133228879118505, 10.14403604765983, -0.010363952340171068)